In [2]:
import numpy as np


In [129]:
def H(node):
    node = np.array(node)
    D_node = node.sum()
    P = node/D_node
    W = D_node/D
    return -(np.sum(W * P * np.log2(P)))

def children_H(children_nodes):
    tmp_h = 0 
    for child_node in children_nodes:
        tmp_h += H(child_node)
    return tmp_h

def get_P(description, target):
    elments = np.unique(description)
    data_P = []
    for el in elments:
        _, cnts = np.unique(target[description==el], return_counts=True)
        data_P.append(cnts)
    return data_P

def get_target_data(description, target):
    elments = np.unique(description)
    target_data = []
    for el in elments:
        # print(orign[data==el])
        target_data.append(target[description==el])
    return target_data

def cal_weighted_var(t):
    t = np.array(t)
    W = t.size/D
    return W*(np.var(t)*(t.size)/(t.size-1)) # var/n-1 (모집단이므로 n-1 사용)

def sum_var(description):
    tmp = 0
    for t in description:
        tmp += cal_weighted_var(t)
    return tmp

def cal_sum_var(data, orign):
    return sum_var(get_target_data(data, orign))


In [138]:
# continuous target features
season = np.array(['winter', 'winter', 'winter', 'spring', 'spring', 'spring', 'summer', 'summer', 'summer', 'autumn', 'autumn', 'autumn'])
work_day = np.array([False, False, True, False, True, True, False, True, True, False, False, True])
rentals = np.array([800, 826, 900, 2100, 4740, 4900, 3000, 5800, 6200, 2910, 2880, 2820])
D = rentals.size # 12

print('---- depth 0 ----')
season_0_0 = cal_sum_var(season, rentals) # 선택
wrk_d_0_0 = cal_sum_var(work_day, rentals)
print(f'season_var: {season_0_0}')
print(f'wrk_d_0_0 : {wrk_d_0_0}')


print('---- depth 1-1 ----') # season == winter
wrk_d_1_1 = np.array([False, False, True])
renta_1_1 = np.array([800, 826, 900])
print(f'wrk_d_1_1 : {cal_sum_var(wrk_d_1_1, renta_1_1)}')

print('---- depth 1-2 ----') # season == spring
wrk_d_1_2 = np.array([False, True, True])
renta_1_2 = np.array([2100, 4740, 4900])


print('---- depth 1-3 ----') # season == summer
wrk_d_1_3 = np.array([False, True, True])
renta_1_3 = np.array([3000, 5800, 6200])


print('---- depth 1-4 ----') # season == autumn
wrk_d_1_4 = np.array([False, False, True])
renta_1_4 = np.array([2910, 2880, 2820])



---- depth 0 ----
season_var: 1379331.3333333335
wrk_d_0_0 : 2551813.3333333335
---- depth 1-1 ----
wrk_d_1_1 : nan
---- depth 1-2 ----
---- depth 1-3 ----
---- depth 1-4 ----


/var/folders/lq/nqfbjmpd3bl434m7j6ktxqn80000gn/T/ipykernel_15691/107694444.py:33: RuntimeWarning: invalid value encountered in scalar divide
  return W*(np.var(t)*(t.size)/(t.size-1)) # var/n-1 (모집단이므로 n-1 사용)


In [102]:
# 가중치 계산이 빠져있다...
stream = np.array([False, True, True, False, False, True, True]).reshape(-1, 1) # (7, 1)
slope = np.array(['steep', 'moderate', 'steep', 'steep', 'flat', 'steep', 'steep']).reshape(-1, 1)
elevation = np.array([300, 1200, 1500, 3000, 3900, 4450, 5000]).reshape(-1, 1)
vegetation = np.array(['chapparal', 'riparian', 'riparian', 'chapparal', 'conifer', 'conifer', 'chapparal']).reshape(-1, 1)
dataset = np.concatenate([stream, slope, elevation, vegetation], axis=1) # (7, 4)

vegetation_P = [3, 2, 2]
D = sum(vegetation_P)
root_H = H(vegetation_P)


get_P(stream, vegetation)
print(children_H(get_P(stream, vegetation)))

# P
stream_P = [[2, 1, 1], [2, 1]]
slope_P = [[3, 1 ,1], [1], [1]]
threshold1 = [[1], [1, 3, 2]] # 750
threshold2 = [[1, 1], [1, 2, 2]] # 1350
threshold3 = [[2, 1], [2, 2]] # 2250
threshold4 = [[2, 3], [2]] # 4175 


print(f'root H : {root_H}')
print(f'stream_P : {children_H(stream_P)}')
print(f'slope_P : {children_H(slope_P)}')
print(f'elevation threshold1 : {children_H(threshold1)}')
print(f'elevation threshold2 : {children_H(threshold2)}')
print(f'elevation threshold3 : {children_H(threshold3)}')
print(f'elevation threshold4 : {children_H(threshold4)}')

print('---IG---')  # 큰 값 선택
print(f'stream_p IG : {root_H - children_H(stream_P)}')
print(f'slope_P IG : {root_H - children_H(slope_P)}')
print(f'elevation threshold1 IG : {root_H - children_H(threshold1)}')
print(f'elevation threshold2 IG : {root_H - children_H(threshold2)}')
print(f'elevation threshold3 IG : {root_H - children_H(threshold3)}')
print(f'elevation threshold4 IG : {root_H - children_H(threshold4)}') # 선택


depth1_H = children_H(threshold4)
print('----- depth 1-1 -----') # data < elevation(4175)
stream_d1_left_P = [[2, 1], [2]]
slope_d1_left_P = [[1], [3, 1]]
threshold1_d1_left_P = [[1], [3, 1]] # 750
threshold2_d1_left_P = [[1, 1], [2, 1]] # 1350
threshold3_d1_left_P = [[2, 1], [2]] # 2250
print(f'stream_d1_left_P          : {children_H(stream_d1_left_P)}')
print(f'slope_d1_left_P           : {children_H(slope_d1_left_P)}')
print(f'threshold1_d1_left_P      : {children_H(threshold1_d1_left_P)}')
print(f'threshold2_d1_left_P      : {children_H(threshold2_d1_left_P)}')
print(f'threshold3_d1_left_P      : {children_H(threshold3_d1_left_P)}')
print(f'stream_d1_left_P IG(select): {depth1_H - children_H(stream_d1_left_P)}') # depth 2(1) 선택
print(f'slope_d1_left_P IG        : {depth1_H - children_H(slope_d1_left_P)}')
print(f'threshold1_d1_left_P IG   : {depth1_H - children_H(threshold1_d1_left_P)}')
print(f'threshold2_d1_left_P IG   : {depth1_H - children_H(threshold2_d1_left_P)}')
print(f'threshold3_d1_left_P IG   : {depth1_H - children_H(threshold3_d1_left_P)}') # depth 2(2)


print('----- depth 1-1 -----') # data > elevation(4175)
stream_d1_right_P = [[1], [1]]
slope_d1_right_P = [[1], [1]]
threshold4_d1_right_P = [[2]] # 4175
print(f'stream_d1_right_P    : {children_H(stream_d1_right_P)}')
print(f'slope_d1_right_P     : {children_H(slope_d1_right_P)}')
print(f'threshold4_d1_right_P: {children_H(threshold4_d1_right_P)}')
print(f'stream_d1_right_P IG : {depth1_H - children_H(stream_d1_right_P)}')
print(f'slope_d1_right_P IG  : {depth1_H - children_H(slope_d1_right_P)}')
print(f'threshold4_d1_right_P IG: {depth1_H - children_H(threshold4_d1_right_P)}')
print('----- depth 1 : right ----- -> complete')


depth2_H = children_H(stream_d1_left_P)
print('----- depth 2(1)-1 -----') # data < elevation(4175) and stream == True
slope_d2_left_P = [[2, 1], [2]]
threshold1_d2_left_P = [[1], [1, 1]] # 750
threshold2_d2_left_P = [[1], [1, 1]] # 1350
threshold3_d2_left_P = [[2], [1]] # 2250
print(f'slope_d2_left_P         : {children_H(slope_d2_left_P)}')
print(f'threshold1_d2_left_P    : {children_H(threshold1_d2_left_P)}')
print(f'threshold2_d2_left_P    : {children_H(threshold2_d2_left_P)}')
print(f'threshold3_d2_left_P    : {children_H(threshold3_d2_left_P)}')
print(f'slope_d2_left_P IG      : {depth2_H - children_H(slope_d2_left_P)}')
print(f'threshold1_d2_left_P IG : {depth2_H - children_H(threshold1_d2_left_P)}')
print(f'threshold2_d2_left_P IG : {depth2_H - children_H(threshold2_d2_left_P)}')
print(f'threshold3_d2_left_P IG : {depth2_H - children_H(threshold3_d2_left_P)}') # 선택

print('----- depth 2(1)-2 -----') # data < elevation(4175) and stream == False
slope_d2_right_P = [[2]]
threshold1_d2_right_P = [[2]] # 750
threshold2_d2_right_P = [[1], [1]] # 1350
threshold3_d2_right_P = [[1], [1]] # 2250
print(f'slope_d2_right_P         : {children_H(slope_d2_right_P)}')
print(f'threshold1_d2_right_P    : {children_H(threshold1_d2_right_P)}')
print(f'threshold2_d2_right_P    : {children_H(threshold2_d2_right_P)}')
print(f'threshold3_d2_right_P    : {children_H(threshold3_d2_right_P)}')
print(f'slope_d2_right_P IG      : {depth2_H - children_H(slope_d2_right_P)}')
print(f'threshold1_d2_right_P IG : {depth2_H - children_H(threshold1_d2_right_P)}')
print(f'threshold2_d2_right_P IG : {depth2_H - children_H(threshold2_d2_right_P)}')
print(f'threshold3_d2_right_P IG : {depth2_H - children_H(threshold3_d2_right_P)}')

print('----- depth 2(2)-1 -----') # data < elevation(4175) and data < elevation(2250)
steam_d2_left_P = [[2], [1]]
slope_d2_left_P = [[1, 1], [1]]
elev1_d2_left_P = [[1], [1, 1]]
elev2_d2_left_P = [[1, 1], [1]]
print(f'steam_d2_left_P         : {children_H(steam_d2_left_P)}')
print(f'slope_d2_left_P         : {children_H(slope_d2_left_P)}')
print(f'elev1_d2_left_P         : {children_H(elev1_d2_left_P)}')
print(f'elev2_d2_left_P         : {children_H(elev2_d2_left_P)}')


print('----- depth 2(2)-2 -----') # data < elevation(4175) and data > elevation(2250)
steam_d2_right_P = [[1], [1]]
slope_d2_right_P = [[2]]
elev1_d2_right_P = [0]
elev2_d2_right_P = [0]
print(f'steam_d2_right_P         : {children_H(steam_d2_right_P)}')
print(f'slope_d2_right_P         : {children_H(slope_d2_right_P)}')



print('----- depth 3-1 -----') # data < elevation(4175) and stream == True  and data > elevation(2250)
print('only ID 7')

print('----- depth 3-2 -----') # data < elevation(4175) and stream == False and data < elevation(2250)
slope_d3_right_P = [[1], [1]]
slope_d3_right_P = [[1], [1]]
elev1_d3_right_P = [[1], [1]]
elev2_d3_right_P = [[1], [1]]

print('----- depth 4 ----- ')

print('elevation(4175) -> stream -> elevation(2250)')

1.2506982145947811
root H : 1.5566567074628228
stream_P : 1.2506982145947811
slope_P : 0.9792504246104776
elevation threshold1 : 1.2506982145947811
elevation threshold2 : 1.3728057820624016
elevation threshold3 : 0.9649839288804956
elevation threshold4 : 0.6935361388961918
---IG---
stream_p IG : 0.30595849286804166
slope_P IG : 0.5774062828523452
elevation threshold1 IG : 0.30595849286804166
elevation threshold2 IG : 0.18385092540042125
elevation threshold3 IG : 0.5916727785823273
elevation threshold4 IG : 0.863120568566631
----- depth 1-1 -----
stream_d1_left_P          : 0.3935553574519241
slope_d1_left_P           : 0.46358749969093305
threshold1_d1_left_P      : 0.46358749969093305
threshold2_d1_left_P      : 0.6792696431662097
threshold3_d1_left_P      : 0.3935553574519241
stream_d1_left_P IG(select): 0.2999807814442677
slope_d1_left_P IG        : 0.22994863920525876
threshold1_d1_left_P IG   : 0.22994863920525876
threshold2_d1_left_P IG   : 0.014266495729982065
threshold3_d1_left